# Estudio de una simulación para averiguar la cantidad de desarrolladores óptima, para un proyecto con tareas de diferente criticidad (altas y bajas).


## Variables Exogenas

### Datos

* **IA**: Intervalo entre Arribos [minutos]
* **TA**: Tiempo de Atencion [minutos]

Estan dados en base a distintas funciones de distribucion de probabilidad, obtenidas mediante _fitting_ de datos reales.


In [1]:
import dataclasses
import sys

from scipy import stats

HV: int = sys.maxsize


def time_between_arrivals() -> int:
    """
    Provides the next arrival the time delta for the next arrival
    
    Returns:
        int: how many minutes until the next arrival
    """
    best_params = {'loc': 0.0, 'scale': 47.291173845125044}

    next_arrival_in = 0

    while next_arrival_in <= 0:
        [next_arrival_in] = stats.halflogistic.rvs(**best_params, size=1)

    return next_arrival_in


def service_time(is_senior: bool = False) -> int:
    """
    Provides the time delta for the next service
    
    Args:
        is_senior: if the service is for a senior developer
    
    Returns:
        int: how many minutes the service will take
    """
    best_params = {'beta': 0.35001800492775303, 'loc': 300.0, 'scale': 3.071132572061014}
    busy_time = 0

    while busy_time <= 0:
        [busy_time] = stats.gennorm.rvs(**best_params, size=1)

    return busy_time * 0.75 if is_senior else busy_time

### Control

* **NPS**: Numero de Programadores Sr
* **NPJ**: Numero de Programadores Jr

In [2]:
@dataclasses.dataclass
class ControlVector:
    """
    Control vector for the simulation
    
    Attributes:
        nps (int): Number of Senior Programmers
        npj (int): Number of Junior Programmers
    """
    nps: int
    npj: int

## Variables Endogenas

### Estado

* **NSH**: Numero de tickets en cola de prioridad alta
* **NSL**: Numero de tickets en cola de prioridad baja

In [3]:
@dataclasses.dataclass
class StatusVector:
    """
    Status vector for the simulation
    
    Attributes:
        nsh (int): Number of high priority tickets
        nsl (int): Number of low priority tickets
    """
    nsh: int
    nsl: int

### Resultados

* **PPS**: Promedio de Permanencia en el Sistema [Dias]
* **PEC**: Promedio de Permanencia en la Cola [Dias]
* **PTO**: Promedio de Tiempo Ocioso [minutos]
* **PTTS**: Porcentaje de Tickets de Baja Prioridad Atendidos por Sr [%]

In [4]:
@dataclasses.dataclass
class ResultVector:
    """
    Result vector for the simulation
    
    Attributes:
        pps (float): Average time in the system
        pec_jr (float): Average time in the queue for Jrs
        pec_sr (float): Average time in the queue for Srs
        pto_jr (list[float]): Average idle time for Jrs
        pto_sr (list[float]): Average idle time for Srs
        ptts (float): Percentage of low priority tickets attended by Sr
    """
    pps: float
    pec_jr: float
    pec_sr: float
    pto_sr: list[float]
    pto_jr: list[float]
    ptts: float

## Tabla de Eventos Indepedientes

| Evento       | Evento Futuro No Condicionado | Evento Futuro Condicionado | Condicion                                 |
|--------------|-------------------------------|----------------------------|-------------------------------------------|
| LLEGADA      | LLEGADA                       | Salida Sr[i]               | NSH <= NPS \|\| (NSL > NPJ && NSH <= NPS) |
| ...          | ...                           | Salida Jr[j]               | NSL <= NPJ                                |
| Salida Sr[i] |                               | Salida Sr[i]               | NSH > NPS \|\| (NSL > NPJ && NSH > NPS)   |
| Salida Jr[j] |                               | Salida Jr[j]               | NSL > NPJ                                 |

## Tabla de Eventos Futuros

TPLL (Tiempo de Llegada del Proximo Ticket), TPSSi (Tiempo de Proxima Salida del Programador Sr i-esimo), TPSJj (Tiempo de Proximo Salida del Programador j-esimo)

In [5]:
@dataclasses.dataclass
class FutureEventsVector:
    """
    Future events vector for the simulation
    
    Attributes:
        tpll (int): Time of the next ticket arrival
        tpss (int): Time of the next Senior Programmer exit
        tpsj (int): Time of the next Junior Programmer exit
    """
    tpll: int
    tpss: list[int]
    tpsj: list[int]

    @staticmethod
    def find_min(future_events: list[int]) -> int:
        """
        Get next event in given list
        
        Args:
            future_events (list[int]): list of future events
        
        Returns:
            int: the index for the nearest event exit
        """
        return future_events.index(min(future_events))

    @staticmethod
    def find_idle(future_events: list[int]) -> int:
        """
        Get the index of the next idle programmer
        
        Args:
            future_events (list[int]): list of future events
        
        Returns:
            int: the index for the nearest idle programmer
        """
        return future_events.index(HV)

## Simulation

In [68]:
import random


def simulate(control: ControlVector, tf: int) -> ResultVector:
    """
    Simulate the system with the given control vector
    
    Args:
        control (ControlVector): the control vector for the simulation
        tf (int): the final time for the simulation
    
    Returns:
        ResultVector: the result vector for the simulation
    """
    # initial conditions
    status = StatusVector(nsh=0, nsl=0)
    future_events = FutureEventsVector(tpll=0, tpss=[HV] * control.nps, tpsj=[HV] * control.npj)
    results = ResultVector(pps=.0, pec_sr=.0, pec_jr=0, pto_sr=[0] * control.nps, pto_jr=[0] * control.npj, ptts=.0)

    # accumulators
    stll, sts = 0, 0
    sto_sr, ito_sr = [0] * control.nps, [0] * control.nps
    sto_jr, ito_jr = [0] * control.npj, [0] * control.npj
    sta_sr = 0
    sta_jr = 0
    tts = 0
    nt_jr = 0
    nt_sr = 0

    # Initialize the simulation clock
    t = 0

    # Run the simulation
    while t < tf or status.nsh > 0 or status.nsl > 0:
        # Get the next event
        tpll = future_events.tpll
        senior_index = future_events.find_min(future_events.tpss)
        jr_index = future_events.find_min(future_events.tpsj)

        # Get the next event time
        if tpll < future_events.tpss[senior_index] and tpll < future_events.tpsj[jr_index]:
            # Event: Arrival

            # Advance the simulation clock
            t = tpll

            # Generate the next arrival
            tpll = time_between_arrivals()
            future_events.tpll = t + tpll

            # Process the arrival of a new ticket
            r = random.random()

            if status.nsh + status.nsl > 30:
                # skip
                continue

            stll += t

            if r <= .41:
                # Process the arrival of a high priority ticket
                status.nsh += 1

                # Check if a Senior Programmer is available
                if status.nsh <= control.nps:
                    sr_programmer = future_events.find_idle(future_events.tpss)
                    sto_sr[sr_programmer] += t - ito_sr[sr_programmer]

                    ta = service_time()

                    # Schedule the next exit of the Senior Programmer
                    future_events.tpss[sr_programmer] = t + ta

                    sta_sr += ta
                    nt_sr += 1
                else:
                    # do nothing - must wait in the queue
                    ...
            else:
                # Process the arrival of a low priority ticket
                status.nsl += 1

                # Check if a Junior Programmer is available
                if status.nsl <= control.npj:
                    jr_programmer = future_events.find_idle(future_events.tpsj)

                    sto_jr[jr_programmer] += t - ito_jr[jr_programmer]

                    # Schedule the next exit of the Junior Programmer
                    ta = service_time()
                    future_events.tpsj[jr_programmer] = t + ta

                    sta_jr += ta
                    nt_jr += 1
                # Check if a Senior Programmer is available
                elif status.nsh < control.nps:
                    # swap the ticket
                    status.nsl -= 1
                    status.nsh += 1

                    sr_programmer = future_events.find_idle(future_events.tpss)
                    sto_sr[sr_programmer] += t - ito_sr[sr_programmer]

                    ta = service_time(is_senior=True)

                    # Schedule the next exit of the Senior Programmer
                    future_events.tpss[sr_programmer] = t + ta

                    sta_sr += ta
                    nt_sr += 1
                    tts += 1
                else:
                    # do nothing - must wait in the queue
                    ...
        elif future_events.tpss[senior_index] < future_events.tpsj[jr_index]:
            # Event: Senior Programmer Exit

            # Advance the simulation clock
            t = future_events.tpss[senior_index]
            sts += t

            status.nsh -= 1

            # Check if all Jrs are busy and there are more tickets to process
            if status.nsl > control.npj and status.nsh < control.nps:
                # swap the ticket
                status.nsl -= 1
                status.nsh += 1

                # Schedule the next exit of the Senior Programmer
                ta = service_time(is_senior=True)
                future_events.tpss[senior_index] = t + ta
                sta_sr += ta

                tts += 1
                nt_sr += 1
            # Check if there are more tickets to process
            elif status.nsh >= control.nps:
                # Schedule the next exit of the Senior Programmer
                ta = service_time()
                future_events.tpss[senior_index] = t + ta

                sta_sr += ta
                nt_sr += 1
            else:
                # Schedule the Senior Programmer to be idle
                future_events.tpss[senior_index] = HV
                ito_sr[senior_index] = t
        else:
            # Event: Junior Programmer Exit
            # Advance the simulation clock
            t = future_events.tpsj[jr_index]
            sts += t
            status.nsl -= 1

            # Check if there are more tickets to process
            if status.nsl >= control.npj:
                # Schedule the next exit of the Junior Programmer
                ta = service_time()
                future_events.tpsj[jr_index] = t + ta
                sta_jr += ta
                nt_jr += 1
            else:
                # Schedule the Junior Programmer to be idle
                future_events.tpsj[jr_index] = HV
                ito_jr[jr_index] = t

        if t >= tf:
            future_events.tpll = HV

    days = 60 * 24

    print(f"# SIMULATION COMPLETED - Case Srs:{control.nps}, Jrs:{control.npj}")
    print(f"Days: {t / days:.2f}")
    print(f"Tickets taken by Srs: {nt_sr} ({tts} Low Priority)")
    print(f"Tickets taken by Jrs: {nt_jr}")
    print(f"Total tickets taken: {nt_sr + nt_jr}")
    print("----------------------------------------")

    # Calculate the results
    results.pps = (sts - stll) / t
    results.pec_sr = (sts - stll - sta_sr) / nt_sr / days  # minutes to days
    results.pec_jr = (sts - stll - sta_jr) / nt_jr / days  # minutes to days
    results.pto_jr = [sto * 100 / t for sto in sto_jr]
    results.pto_sr = [sto * 100 / t for sto in sto_sr]
    results.ptts = (tts / nt_sr) * 100
    return results

In [51]:
### Create Control vectors for combinations of NPS and NPJ from 1 to 5
controls = [ControlVector(nps=nps, npj=npj) for nps in range(2, 8) for npj in range(1, 5)]

In [71]:
simulation_results = []

for control in controls:
    tf = 60 * 24 * 30 * 12 * 20  # 10 years
    results = simulate(control, tf)
    simulation_results.append((control, results))

# SIMULATION COMPLETED - Case Srs:2, Jrs:1
Days: 7201.97
Tickets taken by Srs: 68167 (27831 Low Priority)
Tickets taken by Jrs: 30447
Total tickets taken: 98614
----------------------------------------
# SIMULATION COMPLETED - Case Srs:2, Jrs:2
Days: 7201.41
Tickets taken by Srs: 63964 (12543 Low Priority)
Tickets taken by Jrs: 61120
Total tickets taken: 125084
----------------------------------------
# SIMULATION COMPLETED - Case Srs:2, Jrs:3
Days: 7201.75
Tickets taken by Srs: 61125 (953 Low Priority)
Tickets taken by Jrs: 85302
Total tickets taken: 146427
----------------------------------------
# SIMULATION COMPLETED - Case Srs:2, Jrs:4
Days: 7202.18
Tickets taken by Srs: 60979 (243 Low Priority)
Tickets taken by Jrs: 87303
Total tickets taken: 148282
----------------------------------------
# SIMULATION COMPLETED - Case Srs:3, Jrs:1
Days: 7201.16
Tickets taken by Srs: 103752 (48498 Low Priority)
Tickets taken by Jrs: 30442
Total tickets taken: 134194
------------------------------

In [72]:
for control, results in simulation_results:
    print(f"Control: No. Srs: {control.nps}, No. Jrs: {control.npj}")
    print(f"Results:")
    print(f"PPS: {results.pps:.2f} days")
    print(f"PEC Sr: {results.pec_sr:.2f} days")
    print(f"PEC Jr: {results.pec_jr:.2f} days")
    print(f"PTO Sr: {[round(pto, 2) for pto in results.pto_sr]}%")
    print(f"PTO Jr: {[round(pto, 2) for pto in results.pto_jr]}%")
    print(f"PTTS: {results.ptts:.2f}%")
    print("----------------------------------------")

Control: No. Srs: 2, No. Jrs: 1
Results:
PPS: 29.96 days
PEC Sr: 2.95 days
PEC Jr: 6.85 days
PTO Sr: [0.0, 0.0]%
PTO Jr: [0.0]%
PTTS: 40.83%
----------------------------------------
Control: No. Srs: 2, No. Jrs: 2
Results:
PPS: 28.80 days
PEC Sr: 3.02 days
PEC Jr: 3.16 days
PTO Sr: [0.0, 0.0]%
PTO Jr: [0.0, 0.01]%
PTTS: 19.61%
----------------------------------------
Control: No. Srs: 2, No. Jrs: 3
Results:
PPS: 24.15 days
PEC Sr: 2.61 days
PEC Jr: 1.80 days
PTO Sr: [0.07, 0.11]%
PTO Jr: [5.01, 6.97, 9.69]%
PTTS: 1.56%
----------------------------------------
Control: No. Srs: 2, No. Jrs: 4
Results:
PPS: 22.72 days
PEC Sr: 2.45 days
PEC Jr: 1.64 days
PTO Sr: [0.23, 0.28]%
PTO Jr: [16.63, 23.04, 31.83, 43.28]%
PTTS: 0.40%
----------------------------------------
Control: No. Srs: 3, No. Jrs: 1
Results:
PPS: 27.55 days
PEC Sr: 1.70 days
PEC Jr: 6.28 days
PTO Sr: [0.0, 0.01, 0.01]%
PTO Jr: [0.01]%
PTTS: 46.74%
----------------------------------------
Control: No. Srs: 3, No. Jrs: 2
Result